In [29]:
# Install required packages
!pip install google-generativeai python-dotenv


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [30]:
# Import required libraries
import google.generativeai as genai
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Configure the Gemini API
GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')

if not GEMINI_API_KEY:
    print("Warning: GEMINI_API_KEY not found in environment variables!")
    print("Please make sure you have:")
    print("1. Created a .env file in your project directory")
    print("2. Added GEMINI_API_KEY=your_actual_api_key to the .env file")
else:
    genai.configure(api_key=GEMINI_API_KEY)
    print("Gemini API configured successfully!")

Gemini API configured successfully!


In [31]:
# Initialize the Gemini model
def initialize_gemini_model(model_name="gemini-1.5-flash"):
    """
    Initialize and return a Gemini model instance
    
    Args:
        model_name (str): The name of the Gemini model to use
                         Options: "gemini-1.5-flash", "gemini-1.5-pro", "gemini-1.0-pro"
    
    Returns:
        GenerativeModel: The initialized Gemini model
    """
    try:
        model = genai.GenerativeModel(model_name)
        print(f"{model_name} model initialized successfully!")
        return model
    except Exception as e:
        print(f"Error initializing model: {e}")
        return None

# Initialize the model
model = initialize_gemini_model("gemini-1.5-flash")

gemini-1.5-flash model initialized successfully!


In [32]:
# Check available models
print("Available Gemini Models:")
print("=" * 40)

try:
    available_models = genai.list_models()
    for model_info in available_models:
        print(f"Model: {model_info.name}")
        print(f"  - Display Name: {model_info.display_name}")
        print(f"  - Supported Methods: {model_info.supported_generation_methods}")
        print()
except Exception as e:
    print(f"Error listing models: {e}")

Available Gemini Models:
Model: models/embedding-gecko-001
  - Display Name: Embedding Gecko
  - Supported Methods: ['embedText', 'countTextTokens']

Model: models/gemini-1.5-pro-latest
  - Display Name: Gemini 1.5 Pro Latest
  - Supported Methods: ['generateContent', 'countTokens']

Model: models/gemini-1.5-pro-002
  - Display Name: Gemini 1.5 Pro 002
  - Supported Methods: ['generateContent', 'countTokens', 'createCachedContent']

Model: models/gemini-1.5-pro
  - Display Name: Gemini 1.5 Pro
  - Supported Methods: ['generateContent', 'countTokens']

Model: models/gemini-1.5-flash-latest
  - Display Name: Gemini 1.5 Flash Latest
  - Supported Methods: ['generateContent', 'countTokens']

Model: models/gemini-1.5-flash
  - Display Name: Gemini 1.5 Flash
  - Supported Methods: ['generateContent', 'countTokens']

Model: models/gemini-1.5-flash-002
  - Display Name: Gemini 1.5 Flash 002
  - Supported Methods: ['generateContent', 'countTokens', 'createCachedContent']

Model: models/gemini-1

In [33]:
# Re-initialize the model (fixes the variable overwrite from model listing)
print("Re-initializing the Gemini model...")
model = initialize_gemini_model("gemini-1.5-flash")
print(f"Model type: {type(model)}")
print("Ready to generate content!")

Re-initializing the Gemini model...
gemini-1.5-flash model initialized successfully!
Model type: <class 'google.generativeai.generative_models.GenerativeModel'>
Ready to generate content!


In [34]:
# Helper functions for interacting with Gemini

def generate_text(prompt, model=None, temperature=0.7, max_output_tokens=1000):
    """
    Generate text using the Gemini model
    
    Args:
        prompt (str): The input prompt
        model: The Gemini model instance (uses global model if None)
        temperature (float): Controls randomness (0.0 to 1.0)
        max_output_tokens (int): Maximum number of tokens to generate
    
    Returns:
        str: The generated text response
    """
    if model is None:
        model = globals().get('model')
    
    if model is None:
        return "Error: Model not initialized. Please run the model initialization cell first."
    
    try:
        # Configure generation parameters
        generation_config = genai.types.GenerationConfig(
            temperature=temperature,
            max_output_tokens=max_output_tokens,
        )
        
        # Generate content
        response = model.generate_content(
            prompt,
            generation_config=generation_config
        )
        
        return response.text
        
    except Exception as e:
        return f"Error generating text: {e}"

print("Helper functions defined successfully!")

Helper functions defined successfully!


In [35]:
# Example 1: Simple text generation
print('Type 1: First draft of prompt')

system_prompt = """
    You are an assistant that converts natural language software requirements into formal specifications.

    Instructions:
    - The input will be a short requirement text in plain English, written like an SRS excerpt.
    - The output must contain one or more formal specification blocks.

    Each specification block must include:
    1. Label (e.g., SIGNUP_OK, LOGIN_ERR1)
    2. Precondition: conditions that must hold before the action
    3. API Call: system call in the format action(parameters) → HTTP status code
    4. Postcondition: state of the system after execution, expressed with set/mapping notation (e.g., U' = U[u → p])

    Guidelines:
    - Use compact, mathematical notation:
    - u ∈ dom(U) means “user exists in the user database”
    - u ∉ dom(U) means “user does not exist in the database”
    - U' denotes the updated user database
    - For each error case, create a separate block with the same structure (ERR1, ERR2, etc.).
    - Postcondition for errors should typically leave the system state unchanged (U' = U).
    - Maintain consistent, structured formatting.

    Example Input:
    the user id must not exist in the userdb for sign up to succeed
    the user id must exist in the userdb and the password provided must match the corresponding password in the db

    Example Output:
    SIGNUP_OK
    Precondition: u ∉ dom(U)
    API Call: signup(u,p) → HTTP OK 200
    Postcondition: U' = U[u → p]

    SIGNUP_ERR1
    Precondition: u ∈ dom(U)
    API Call: signup(u,p) → HTTP 409
    Postcondition: U' = U

    LOGIN_OK
    Precondition: u ∈ dom(U) ∧ U[u] = p
    API Call: login(u,p) → HTTP OK 200
    Postcondition: session(u) = active

    LOGIN_ERR1
    Precondition: u ∉ dom(U)
    API Call: login(u,p) → HTTP 404
    Postcondition: U' = U
"""

user_prompt = """
    Convert the following natural language requirements into formal specifications:

    - A book can be borrowed only if it exists in the library catalog.
    - The book must not already be checked out.
    - If both conditions hold, the system marks the book as checked out.
    - If the book does not exist in the catalog, the system returns an error.
    - If the book is already checked out, the system returns an error.
"""


# user_prompt = """
#     Convert the following natural language requirements into formal specifications:

#     - The user must exist in the userdb for password reset to proceed.
#     - The reset token provided must be valid.
#     - If both conditions hold, the system updates the password for that user in the userdb.
#     - If the user does not exist, the system must return an error.
#     - If the reset token is invalid, the system must return an error.
# """


payload = {
    "system": system_prompt,
    "user": user_prompt
}

Type 1: First draft of prompt


In [36]:
full_prompt = f"{system_prompt}\n{user_prompt}"

response = generate_text(full_prompt, model=model)

print(response)

BORROW_OK
Precondition: b ∈ dom(B) ∧ B[b].available = true
API Call: borrow(b, u) → HTTP OK 200
Postcondition: B' = B[b → {available: false, borrower: u}]

BORROW_ERR1
Precondition: b ∉ dom(B)
API Call: borrow(b, u) → HTTP 404
Postcondition: B' = B

BORROW_ERR2
Precondition: b ∈ dom(B) ∧ B[b].available = false
API Call: borrow(b, u) → HTTP 409
Postcondition: B' = B

Where:

- B represents the library catalog (a mapping from book ID 'b' to book information, including availability).
- u represents the user ID.
- B[b].available represents the availability status of book 'b' (true or false).
- B[b → {available: false, borrower: u}] updates the catalog to mark book 'b' as unavailable and assigns it to user 'u'.


